<a href="https://colab.research.google.com/github/asmis11/video-content-moderation/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
uploaded = files.upload()

Saving tikharm_dataset.zip to tikharm_dataset.zip


In [8]:
import zipfile

base_path = "/content/tikharm_dataset.zip"

with zipfile.ZipFile(base_path, 'r') as zip_ref:
    zip_ref.extractall("/content/tikharm_dataset")

extracted_dir = "/content/tikharm_dataset"
print(os.listdir(extracted_dir))

['tikharm_dataset']


In [9]:
!pip install torch torchvision torchaudio torchvggish moviepy ffmpeg-python --quiet

import os
import glob
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
import torchaudio
import torchvggish
import moviepy.editor as mp
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision.models.video import r3d_18
from tqdm import tqdm

  if event.key is 'enter':



3D CNN's

In [42]:
def extract_frames(video_path, num_frames=8):
    vidcap = cv2.VideoCapture(video_path)
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames <= 0:
        raise Exception(f"Video {video_path} is empty or corrupted.")
    frame_idxs = np.linspace(0, total_frames-1, num_frames).astype(int)
    frames = []
    frame_count = 0
    success = True
    while success:
        success, image = vidcap.read()
        if success and frame_count in frame_idxs:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (112,112))
            frames.append(image)
        frame_count += 1
    vidcap.release()
    return frames

In [43]:
class_map = {
    "adult": 0,
    "harmful": 1,
    "safe": 2,
    "suicide": 3
}

class VideoDataset(Dataset):
    def __init__(self, base_folder, num_frames=8):
        self.video_paths = []
        self.labels = []
        for class_folder, label in class_map.items():
            folder_path = os.path.join(base_folder, class_folder)
            videos = glob.glob(folder_path + '/*.mp4')
            self.video_paths += videos
            self.labels += [label] * len(videos)
        self.num_frames = num_frames
        print(f"✅ Found {len(self.video_paths)} videos.")

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]

        frames = extract_frames(video_path, num_frames=self.num_frames)
        frame_tensors = []
        for img in frames:
            img = torch.tensor(img).permute(2,0,1).float() / 255.0
            frame_tensors.append(img)

        video_input = torch.stack(frame_tensors, dim=1)  # [C, T, H, W]
        return video_input, label

In [44]:
class VideoOnlyModel(nn.Module):
    def __init__(self):
        super(VideoOnlyModel, self).__init__()
        self.backbone = r3d_18(pretrained=True)
        self.backbone.fc = nn.Linear(512, 4)

    def forward(self, x):
        return torch.softmax(self.backbone(x), dim=1)

In [45]:
BASE_PATH = '/content/tikharm_dataset/tikharm_dataset/'
TRAIN_PATH = os.path.join(BASE_PATH, "train")

batch_size = 2

train_dataset = VideoDataset(TRAIN_PATH)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = VideoOnlyModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for video_x, labels in tqdm(train_loader):
        video_x = video_x.to(device)
        labels = labels.to(device)

        outputs = model(video_x)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

print("✅ Training Complete!")

✅ Found 35 videos.


100%|██████████| 18/18 [04:10<00:00, 13.89s/it]


Epoch [1/3], Loss: 1.3754


100%|██████████| 18/18 [04:07<00:00, 13.72s/it]


Epoch [2/3], Loss: 1.1652


100%|██████████| 18/18 [04:06<00:00, 13.71s/it]

Epoch [3/3], Loss: 1.0867
✅ Training Complete!


In [47]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    test_video_path = os.path.join('/content', filename)
    print(f"✅ Uploaded Test Video: {test_video_path}")

categories = ['Adult Content', 'Harmful Content', 'Safe', 'Suicide']

def predict_video(video_path):
    frames = extract_frames(video_path, num_frames=8)
    frame_tensors = []
    for img in frames:
        img = torch.tensor(img).permute(2,0,1).float() / 255.0
        frame_tensors.append(img)

    video_input = torch.stack(frame_tensors, dim=1)
    video_input = video_input.unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        preds = model(video_input)

    pred_idx = preds.argmax(dim=1).item()
    confidence = preds.max().item()

    pred_category = categories[pred_idx]
    safe_or_unsafe = "Safe" if pred_category == "Safe" else "Unsafe"

    notes = {
        'Adult Content': "Contains adult/inappropriate material",
        'Harmful Content': "Contains dangerous/violent activities",
        'Safe': "Content appropriate for all audiences",
        'Suicide': "Self-harm or suicide discussion"
    }[pred_category]

    print(f"🎥 Video: {os.path.basename(video_path)}")
    print(f"📚 Category: {pred_category}")
    print(f"🛡️ Safe/Unsafe: {safe_or_unsafe}")
    print(f"🔢 Confidence: {confidence:.2f}")
    print(f"📝 Notes: {notes}")

# Predict the uploaded video
predict_video(test_video_path)


Saving dylanpawson_7174403759286635778.mp4 to dylanpawson_7174403759286635778.mp4
✅ Uploaded Test Video: /content/dylanpawson_7174403759286635778.mp4
🎥 Video: dylanpawson_7174403759286635778.mp4
📚 Category: Safe
🛡️ Safe/Unsafe: Safe
🔢 Confidence: 0.60
📝 Notes: Content appropriate for all audiences


In [48]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    test_video_path = os.path.join('/content', filename)
    print(f"✅ Uploaded Test Video: {test_video_path}")

categories = ['Adult Content', 'Harmful Content', 'Safe', 'Suicide']

def predict_video(video_path):
    frames = extract_frames(video_path, num_frames=8)
    frame_tensors = []
    for img in frames:
        img = torch.tensor(img).permute(2,0,1).float() / 255.0
        frame_tensors.append(img)

    video_input = torch.stack(frame_tensors, dim=1)  # [C, T, H, W]
    video_input = video_input.unsqueeze(0).to(device)  # [1, C, T, H, W]

    model.eval()
    with torch.no_grad():
        preds = model(video_input)

    pred_idx = preds.argmax(dim=1).item()
    confidence = preds.max().item()

    pred_category = categories[pred_idx]
    safe_or_unsafe = "Safe" if pred_category == "Safe" else "Unsafe"

    notes = {
        'Adult Content': "Contains adult/inappropriate material",
        'Harmful Content': "Contains dangerous/violent activities",
        'Safe': "Content appropriate for all audiences",
        'Suicide': "Self-harm or suicide discussion"
    }[pred_category]

    print(f"🎥 Video: {os.path.basename(video_path)}")
    print(f"📚 Category: {pred_category}")
    print(f"🛡️ Safe/Unsafe: {safe_or_unsafe}")
    print(f"🔢 Confidence: {confidence:.2f}")
    print(f"📝 Notes: {notes}")

# Predict the uploaded video
predict_video(test_video_path)


Saving dinhthuyvlog_7254957847811886342.mp4 to dinhthuyvlog_7254957847811886342.mp4
✅ Uploaded Test Video: /content/dinhthuyvlog_7254957847811886342.mp4
🎥 Video: dinhthuyvlog_7254957847811886342.mp4
📚 Category: Adult Content
🛡️ Safe/Unsafe: Unsafe
🔢 Confidence: 0.44
📝 Notes: Contains adult/inappropriate material


FUSION MODEL

In [53]:
import os
import glob
import numpy as np
import pandas as pd
import librosa
import torchaudio
import torchvggish
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [54]:
def collect_metadata(dataset_path, split='train', max_safe=10, max_unsafe=30):
    rows = []
    label_map = {'safe': 0, 'adult': 1, 'harmful': 1, 'suicide': 1}  # binary
    for label in os.listdir(os.path.join(dataset_path, split)):
        folder = os.path.join(dataset_path, split, label)
        if not os.path.isdir(folder): continue
        count = 0
        max_limit = max_safe if label == 'safe' else max_unsafe // 3
        for f in os.listdir(folder):
            if f.endswith(".mp4"):
                rows.append({
                    'video_id': f,
                    'path': os.path.join(folder, f),
                    'category': label,
                    'notes': f"Auto-generated notes for {label}",
                    'label': label_map.get(label, 1)
                })
                count += 1
                if count >= max_limit:
                    break
    return pd.DataFrame(rows)

In [55]:
def extract_audio_confidence(path):
    try:
        y, sr = librosa.load(path, sr=None, duration=5)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        return round(np.mean(mfcc), 2) / 100
    except:
        return 0.5

def extract_visual_confidence():
    return round(np.random.uniform(0.6, 0.95), 2)

def extract_text_confidence(notes, vectorizer):
    vec = vectorizer.transform([notes]).toarray()
    return round(np.mean(vec), 2)

def generate_notes(category, label, confidence):
    if label == "safe":
        return "No harmful indicators found." if confidence > 0.85 else "Likely safe, review recommended."
    elif category == "adult":
        return "May contain adult or explicit content."
    elif category == "harmful":
        return "Shows violence or dangerous activity."
    elif category == "suicide":
        return "Possible suicide-related content."
    return "Potentially unsafe."


In [56]:
base_path = "/content/tikharm_dataset/tikharm_dataset"
df_train = collect_metadata(base_path, split="train", max_safe=10, max_unsafe=30)

vectorizer = TfidfVectorizer()
vectorizer.fit(df_train['notes'])
df_train['text_conf'] = df_train['notes'].apply(lambda x: extract_text_confidence(x, vectorizer))
df_train['audio_conf'] = df_train['path'].apply(extract_audio_confidence)
df_train['visual_conf'] = df_train['path'].apply(lambda x: extract_visual_confidence())

X = df_train[['audio_conf', 'visual_conf', 'text_conf']]
y = df_train['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
fusion_model = LogisticRegression()
fusion_model.fit(X_train, y_train)
print("✅ Fusion Model Trained!")

  y, sr = librosa.load(path, sr=None, duration=5)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)

  y, sr = librosa.load(path, sr=None, duration=5)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)

  y, sr = librosa.load(path, sr=None, duration=5)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)

  y, sr = librosa.load(path, sr=None, duration=5)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)

  y, sr = librosa.load(path, sr=None, duration=5)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, dur

✅ Fusion Model Trained!


In [57]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded:
    test_path = f"/content/{filename}"

    audio_conf = extract_audio_confidence(test_path)
    visual_conf = extract_visual_confidence()
    note = "Auto-generated test video analysis"
    text_conf = extract_text_confidence(note, vectorizer)

    X_new = [[audio_conf, visual_conf, text_conf]]
    pred = fusion_model.predict_proba(X_new)[0]

    confidence = round(float(np.max(pred)), 2)
    label = "unsafe" if pred[1] > 0.5 else "safe"
    notes = generate_notes("unknown", label, confidence)

    print(f"\n🎥 Video: {filename}")
    print(f"🛡️ Prediction: {label.upper()}")
    print(f"🔢 Confidence: {confidence}")
    print(f"📝 Notes: {notes}")

  y, sr = librosa.load(path, sr=None, duration=5)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)

  warnings.warn(



Saving capnyx_7240168632532126994.mp4 to capnyx_7240168632532126994.mp4

🎥 Video: capnyx_7240168632532126994.mp4
🛡️ Prediction: UNSAFE
🔢 Confidence: 0.85
📝 Notes: Potentially unsafe.
